# Environment setting
環境設定

In [1]:
import os
import requests
from pathlib import Path


# determine branch, default is main
branch = "main"

# Check if running in Google Colab
is_colab = "COLAB_GPU" in os.environ

if is_colab:
    # Download the utils.py file from GitHub
    utils_url = (
        f"https://raw.githubusercontent.com/nics-tw/petsard/{branch}/demo/utils.py"
    )
    response = requests.get(utils_url)

    if response.status_code == 200:
        # Save the utils.py file
        with open("utils.py", "w") as f:
            f.write(response.text)

        # Create an empty __init__.py
        Path("__init__.py").touch()
    else:
        raise RuntimeError(
            f"Failed to download utils.py. Status code: {response.status_code}"
        )

In [2]:
# Now import and run the setup
from utils import (
    get_yaml_path,
    setup_environment,
)


setup_environment(
    is_colab,
    branch,
    benchmark_data=[
        "adult-income",
    ],
)

Looking in links: /var/folders/lb/lwjvbr314wj7bt1k63plw4bw0000gn/T/tmpa9h_jpri
Obtaining file:///Users/justyn.chen/Library/CloudStorage/Dropbox/5_Career%20%E5%B7%A5%E4%BD%9C/20231016_NICS%20%E8%B3%87%E5%AE%89%E9%99%A2/3_%E5%B7%A5%E8%97%9D%EF%BC%9APETsARD/petsard
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for petsard (pyproject.toml): started
  Building editable for petsard (pyproject.toml): finished with status 'done'

In [3]:
from petsard import Executor

# YAML Configuration for PETsARD
PETsARD 的 YAML 設定

## Data Constraining
資料約束

In [4]:
yaml_file_case: str = "data-constraining.yaml"

yaml_path_case: str = get_yaml_path(
    is_colab=is_colab,
    yaml_file=yaml_file_case,
    branch=branch,
)

Configuration content:
---
Loader:
  data:
    filepath: 'benchmark/adult-income.csv'
Preprocessor:
  demo:
    method: 'default'
Synthesizer:
  demo:
    method: 'default'
Postprocessor:
  demo:
    method: 'default'
Constrainer:
  demo:
    nan_groups:
      # Delete entire row when workclass is NA
      workclass: 'delete'
      # Set income to NA if occupation is NA
      occupation:
        'erase':
          - 'income'
      # Copy educational-num value to age when educational-num exists but age is NA
      age:
        'copy':
          'educational-num'
    field_constraints:
      - "age >= 18 & age <= 65"
      - "hours-per-week >= 20 & hours-per-week <= 60"
    field_combinations:
      -
        - {'education': 'income'}
        - {'Doctorate': ['>50K'], 'Masters': ['>50K', '<=50K']}
Reporter:
  output:
    method: 'save_data'
    source: 'Constrainer'
...


### Execution and Result
執行與結果

In [5]:
# Initialize and run executor
exec_case = Executor(config=yaml_path_case)
exec_case.run()

Now is petsard_Loader[data]_Preprocessor[demo]_Synthesizer[demo]_Postprocessor[demo]_Constrainer[demo] save to csv...


In [6]:
exec_case.get_result()[
    "Loader[data]_Preprocessor[demo]_Synthesizer[demo]_Postprocessor[demo]_Constrainer[demo]_Reporter[output]"
][
    "Loader[data]_Preprocessor[demo]_Synthesizer[demo]_Postprocessor[demo]_Constrainer[demo]"
]

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,50.918204,Private,140805.923309,Some-college,10.350418,Married-civ-spouse,Prof-specialty,Own-child,White,Male,0.0,0.0,51.347187,United-States,<=50K
1,63.038640,State-gov,357832.768401,11th,9.934143,Married-civ-spouse,Prof-specialty,Husband,White,Male,0.0,0.0,40.751562,United-States,>50K
2,31.781370,?,271431.457340,HS-grad,7.743993,Never-married,Transport-moving,Own-child,Black,Female,0.0,0.0,43.972300,United-States,<=50K
3,48.964596,Private,187883.248724,Some-college,13.929939,Married-civ-spouse,Adm-clerical,Husband,White,Male,0.0,0.0,47.812537,United-States,<=50K
4,31.794678,Private,301282.975291,Some-college,8.833515,Widowed,?,Wife,White,Female,0.0,0.0,36.089767,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28553,48.785775,Private,102247.638616,Assoc-voc,13.751840,Divorced,?,Husband,White,Male,0.0,0.0,38.676404,United-States,<=50K
28554,35.664574,Private,89419.445915,Some-college,8.578969,Never-married,Tech-support,Own-child,White,Male,0.0,0.0,36.377865,United-States,<=50K
28555,45.933704,Private,271121.800698,HS-grad,10.023192,Divorced,?,Not-in-family,White,Female,0.0,0.0,33.303572,United-States,<=50K
28556,49.221063,Self-emp-not-inc,218594.190982,Bachelors,13.461517,Married-civ-spouse,?,Husband,White,Male,0.0,0.0,38.495524,India,<=50K
